In [1]:
import pandas as pd
import regex as re


In [2]:
df_player = pd.read_csv('Resources/players.csv')
df_player_att = pd.read_csv('Resources/players_attributes.csv')
df_team = pd.read_csv('Resources/team.csv')
df_team_att = pd.read_csv('Resources/team_attributes.csv')
df_fifa_players = pd.read_csv('Resources/fifa_players.csv')
df_player_gamestats = pd.read_csv('Resources/final_data.csv')



In [20]:
#Getting distinct values based on latest date of recorded statistics of each Team.
df_team_att['date'] = pd.to_datetime(df_team_att['date'])
df_team_att['date'].dt.date
df_team_att

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,1454,15005,10000,2011-02-22,52,Balanced,NaN,Little,52,Mixed,...,53,Normal,Organised,46,Medium,48,Press,53,Normal,Cover
1454,1455,15005,10000,2012-02-22,54,Balanced,NaN,Little,51,Mixed,...,50,Normal,Organised,44,Medium,55,Press,53,Normal,Cover
1455,1456,15005,10000,2013-09-20,54,Balanced,NaN,Little,51,Mixed,...,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover
1456,1457,15005,10000,2014-09-19,54,Balanced,42.0,Normal,51,Mixed,...,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover


In [55]:
# Merging Team information data with Team Attibutes
df_team_combined = pd.merge(df_team, df_team_att, how='inner', on='team_api_id')

# Filtering Data to show only results of the most recent date
latest_indices = df_team_combined.groupby('team_long_name')['date'].idxmax()
latest_team_df = df_team_combined.loc[latest_indices]

# Replacing Null Values using Fillna , Replacing Buildup play dribbling with 0 where class is "Little"
latest_team_df['buildUpPlayDribbling'].fillna(0, inplace=True)

C:\Users\rdesh\AppData\Local\Temp\ipykernel_41676\2867011828.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  latest_team_df['buildUpPlayDribbling'].fillna(0, inplace=True)


In [3]:
# Defining function to extract the last name from dataframes to have a common element for merging. 
def extract_last_name(name):
    # Use regex to find the last word in the string, which is assumed to be the last name
    last_name = re.search(r'(\w+)$', name)
    
    # Return the last name if a match is found
    if last_name:
        return last_name.group(1)
    else:
        return None

In [6]:
# This Df is a complete set with progressional growth over the years of each player. 
df_player_comp = pd.merge(df_player,df_player_att, how='left', on='player_api_id' ) 

# Getting distinct values based on latest date of recorded statistics of each player. 
df_player_comp['date'] = pd.to_datetime(df_player_comp['date'])
latest_indices = df_player_comp.groupby('player_name')['date'].idxmax()
latest_value = df_player_comp.loc[latest_indices]

latest_value['birthday'] = pd.to_datetime(latest_value['birthday'])
latest_value['birthday'] = latest_value['birthday'].dt.date

In [7]:
# Using function to extract last name using Regex. This way we have a common element in both datasets and can use that to merge. 
latest_value['last_name'] = latest_value['player_name'].apply(extract_last_name)
latest_value


,id_x,player_api_id,player_name,player_fifa_api_id_x,birthday,height,weight,id_y,player_fifa_api_id_y,date,...,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,last_name
0,1,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,1,218353,2016-02-18,...,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,Appindangoye
5,2,155782,Aaron Cresswell,189615,1989-12-15,170.18,146,6,189615,2016-04-21,...,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,Cresswell
38,3,162549,Aaron Doran,186170,1991-05-13,170.18,163,39,186170,2016-01-07,...,61.0,23.0,22.0,24.0,16.0,11.0,12.0,9.0,13.0,Doran
64,4,30572,Aaron Galindo,140161,1982-05-08,182.88,198,65,140161,2016-04-21,...,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0,Galindo
87,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,45.0,75.0,73.0,71.0,8.0,6.0,16.0,12.0,11.0,Hughes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183953,11073,36491,Zsolt Low,111191,1979-04-29,180.34,154,183954,111191,2011-02-22,...,61.0,67.0,71.0,59.0,15.0,13.0,5.0,6.0,6.0,Low
183960,11074,35506,Zurab Khizanishvili,47058,1981-10-06,185.42,172,183961,47058,2011-02-22,...,45.0,65.0,66.0,65.0,14.0,7.0,15.0,10.0,12.0,Khizanishvili
183968,11075,39902,Zvjezdan Misimovic,102359,1982-06-05,180.34,176,183969,102359,2012-08-31,...,78.0,22.0,31.0,30.0,9.0,13.0,8.0,15.0,5.0,Misimovic
41617,2483,11833,de Oliveira Cleber Monteiro,163001,1980-05-23,182.88,170,41618,163001,2012-02-22,...,51.0,53.0,64.0,61.0,15.0,15.0,15.0,12.0,14.0,Monteiro


In [8]:
# Using function to extract last name using Regex. This way we have a common element in both datasets and can use that to merge. 
df_fifa_players.rename(columns= {'birth_date' : 'birthday'}, inplace=True)
df_fifa_players['last_name'] = df_fifa_players['name'].apply(extract_last_name)

In [9]:
# combining dataframes to cummalate all relevant columns 
df_combined = pd.merge(latest_value, df_fifa_players, how= 'inner', on='last_name')

In [100]:
# Removed Duplicate and Null values within the dataframe.
df_combined.drop_duplicates(subset='player_name', inplace=True)
df_cleaned = df_combined.dropna(subset=['wage_euro'])
df_cleaned = df_combined.dropna(subset=['value_euro'])
df_cleaned = df_combined.dropna(subset=['volleys_x'])
df_cleaned = df_combined.dropna(subset=['curve_x'])
df_cleaned = df_combined.dropna(subset=['agility_x'])
df_cleaned = df_combined.dropna(subset=['balance_x'])
df_cleaned = df_combined.dropna(subset=['jumping_x'])

In [101]:
# Converting Birthday Colum to datetime format. 
df_cleaned['birthday_x'] = pd.to_datetime(df_cleaned['birthday_x'])
df_cleaned['birthday_x'].dt.date

C:\Users\rdesh\AppData\Local\Temp\ipykernel_41676\3764648051.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['birthday_x'] = pd.to_datetime(df_cleaned['birthday_x'])


0        1989-12-15
1        1979-11-08
6        1986-09-04
11       1987-04-16
15       1987-10-28
            ...    
18806    1988-10-16
18807    1989-01-01
18808    1979-04-03
18819    1980-05-23
18823    1982-10-06
Name: birthday_x, Length: 5550, dtype: object

In [109]:
# Removed Suffixes ('_x' and '_y') created by the merges 
df_cleaned.rename(columns=lambda x: x.replace('_x', '').replace('_y', ''), inplace=True)
df_cleaned

C:\Users\rdesh\AppData\Local\Temp\ipykernel_41676\3609353918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns=lambda x: x.replace('_x', '').replace('_y', ''), inplace=True)


,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,id,player_fifa_api_id,date,...,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle
0,2,155782,Aaron Cresswell,189615,1989-12-15,170.18,146,6,189615,2016-04-21,...,62,73,75,60,68,59,71,72,78,79
1,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,30,70,65,41,35,38,49,68,70,64
6,6,27316,Aaron Hunt,158138,1986-09-04,182.88,161,113,158138,2016-04-28,...,50,60,65,65,54,48,67,66,69,67
11,8,30895,Aaron Lennon,152747,1987-04-16,165.10,139,147,152747,2015-10-16,...,57,57,52,73,73,62,77,56,50,43
15,10,101042,Aaron Meijers,188621,1987-10-28,175.26,170,180,188621,2015-12-03,...,58,72,65,66,69,56,75,64,69,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18806,11064,107281,Zoltan Stieber,179712,1988-10-16,175.26,148,183823,179712,2016-01-21,...,63,41,31,67,73,65,69,44,31,32
18807,11070,282473,Zouhair Feddal,205705,1989-01-01,190.50,172,183904,205705,2016-02-04,...,56,82,79,48,47,48,69,78,83,78
18808,11071,26357,Zoumana Camara,2488,1979-04-03,182.88,168,183925,2488,2015-03-06,...,55,42,23,62,53,58,59,15,18,11
18819,2483,11833,de Oliveira Cleber Monteiro,163001,1980-05-23,182.88,170,41618,163001,2012-02-22,...,26,65,68,26,29,40,63,61,57,60


In [15]:
#exporting Dataframes to Csv 
df_player.to_csv('Resources/players.csv', index=False)
df_player_att.to_csv('Resources/players_attributes.csv', index=False)
df_team.to_csv('Resources/team.csv', index=False)
df_team_att.to_csv('Resources/team_attributes.csv', index=False)
df_cleaned.to_csv('Resources/cleaned_data.csv', index=False)